# Is my data fit for use - Hive

The SILO data has already been loaded into Hive and HDFS for you. As you will recall from earlier pracs, once the data is in Hive we can use SQL to ask a variety of different questions.

To use Hive, SSH into the data7001 node with your UQ username and type `hive` at the prompt.

Let's use the `show tables` command to see the available tables:

```
hive> show tables;
OK
mdata
silo
Time taken: 0.242 seconds, Fetched: 2 row(s)
```

You'll see that there are two tables ready to be queried, one called `mdata` and one called `silo`.

What format are these tables in? We can use the `desc` command to find out:

```
hive> desc mdata;
OK
fname                   string                                      
latitude                float                                       
elevation               float                                       
id                      int                                         
longitude               float                                       
name                    string                                      
Time taken: 0.42 seconds, Fetched: 6 row(s)
hive> desc silo;
OK
date1                   int                                         
day                     int                                         
date2                   string                                      
tmax                    float                                       
smx                     int                                         
tmin                    float                                       
smn                     int                                         
rain                    float                                       
srn                     int                                         
evap                    float                                       
sev                     int                                         
radn                    float                                       
ssl                     int                                         
vp                      float                                       
svp                     int                                         
rhmaxt                  float                                       
rhmint                  float                                       
fa056                   float                                       
ascepm                  float                                       
mlake                   float                                       
mpot                    float                                       
mact                    float                                       
mwet                    float                                       
span                    float                                       
ssp                     int                                 
evsp                    float                                       
ses                     int                                         
mslpres                 float                                       
sp                      int                                         
Time taken: 0.414 seconds, Fetched: 29 row(s)
```

From the previous part of the prac, we noticed that picking a dataset at random gave us a lot of variability in how many data points were interpolated vs recorded. Let's see if we can find a dataset where most of the Maximum Temperature values are actual recorded values rather than Interpolated. Recall the table from last week the various source codes:

| source code | description
| ---------- |
| 0 | Station data, as supplied by Bureau |
| 13 | Deaccumulated using nearby station |
| 15 | Deaccumulated using interpolated data |
| 23 | Nearby station, data from BoM | 
| 25 | interpolated daily observations |
| 26 | synthetic pan evaporation (only used for Ssp) |
| 35 | interpolated from daily observations using anomaly interpolation method for CLIMARC data | 
| 75 | interpolated long term average | 

In this case, for each *individual file* we want to count how many entries exist where 'smx' (source of max temperature) is equal to 0. To do this, we would need to read every line of each individual file, check the source column and sum the refrequencies.

We could do this with a simple python script, but given how much data we have to parse, a faster solution may be to use Hive to distribute the process across several machines. For a simple GROUP BY query, it also saves us having to write a custom parsing script, since the data is already in Hive ready to be queried.

To run this query in Hive, we can use the following to submit a Map Reduce job that counts the frequencies for each file:

```
hive> select count(*) as cnt, INPUT__FILE__NAME from silo where smx =
'0' GROUP BY INPUT__FILE__NAME ORDER BY cnt desc LIMIT 10;

46674 hdfs://node1.fabric.zones.eait.uq.edu.au:8020/shared/silo/66062_UQ.dat
45552 hdfs://node1.fabric.zones.eait.uq.edu.au:8020/shared/silo/26026_UQ.dat
44455 hdfs://node1.fabric.zones.eait.uq.edu.au:8020/shared/silo/44021_UQ.dat
44082 hdfs://node1.fabric.zones.eait.uq.edu.au:8020/shared/silo/38003_UQ.dat
43968 hdfs://node1.fabric.zones.eait.uq.edu.au:8020/shared/silo/30045_UQ.dat
41333 hdfs://node1.fabric.zones.eait.uq.edu.au:8020/shared/silo/39039_UQ.dat
40960 hdfs://node1.fabric.zones.eait.uq.edu.au:8020/shared/silo/74128_UQ.dat
39988 hdfs://node1.fabric.zones.eait.uq.edu.au:8020/shared/silo/9518_UQ.dat
39412 hdfs://node1.fabric.zones.eait.uq.edu.au:8020/shared/silo/56017_UQ.dat
39345 hdfs://node1.fabric.zones.eait.uq.edu.au:8020/shared/silo/9534_UQ.dat
```

There's one new column you may not be familiar with which is Hive specific - `INPUT__FILE__NAME`. Since Hive is just a front-end for performing work on actual files in HDFS, you can use the virtual column name `INPUT__FILE__NAME` to pull the actual file name the data is sourced from. 

In our case, we performed a count of every match where smx = 0, and grouped the results by the filename. **Also note the `LIMIT 10` at the end**. Adding a LIMIT is important when working with files this size - if you print out hundreds and thousands of lines of data you will be overloaded with output, it's best to limit the results so they are easier to manage and understand. This, along with `ORDER BY` which makes sure it prints in descending order of the result of `count(*)`.

From these results, we can see that the `66062_UQ.dat` file has the highest frequency of Station Data. We can query the 'mdata' table to find out more about this dataset:

```
hive> select * from mdata where fname='66062_UQ.dat';
OK
66062_UQ.dat    -33.8607        39.0    66062   151.205 SYDNEY (OBSERVATORY HILL)
Time taken: 0.697 seconds, Fetched: 1 row(s)
```

So it looks like this dataset is located in Sydney.

What if we want to find all the datasets that are close to our location of interest in `SYDNEY (OBSERVATORY HILL)`? As all the location data is contained within a small, single metadata file, let's go back to Python for a moment to see if we can find out which observatories are closest to our data point of interest.

In [ ]:
import pandas as pd
from vincenty import vincenty

metadata = pd.read_csv("all_metadata.csv")

to_sort = []
base_coords = (-33.8607, 151.205) #co-ordinates for id: 66062
for i in range(0, len(metadata)):
    to_sort.append(
        (metadata['id'][i],
         vincenty(base_coords, (metadata['latitude'][i], metadata['longitude'][i]),miles=False) )
    )
    
to_sort.sort(key=lambda tup: tup[1])
to_sort[:20]

The above code reads from the `all_metadata` file and uses the Vincenty's Forumlae (https://en.wikipedia.org/wiki/Vincenty%27s_formulae) to determine the distance between our original point of interest (-33.8607, 151.205) and each other position contained within our dataset. We then list the first 20 sorted elements by distance in kilometers to find our closest datasets.